# Prepare Dataset for Model Training and Evaluating
## Amazon Customer Reviews Dataset


Amazon Customer Reviews Dataset
https://s3.amazonaws.com/amazon-reviews-pds/readme.html

Over 130+ million customer reviews are available to researchers as part of this release. 

Schema

* marketplace: 2-letter country code (in this case all "US").
* customer_id: Random identifier that can be used to aggregate reviews written by a single author.
* review_id: A unique ID for the review.
* product_id: The Amazon Standard Identification Number (ASIN). 
* product_parent: The parent of that ASIN. Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
* product_title: Title description of the product.
* product_category: Broad product category that can be used to group reviews (in this case digital videos).
* star_rating: The review's rating (1 to 5 stars).
* helpful_votes: Number of helpful votes for the review.
* total_votes: Number of total votes the review received.
* vine: Was the review written as part of the Vine program?
* verified_purchase: Was the review from a verified purchase?
* review_headline: The title of the review itself.
* review_body: The text of the review.
* review_date: The date the review was written.

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

## Download
Let's start by retrieving a subset of the Amazon Customer Reviews dataset.

In [ ]:
!aws s3 cp 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz' ./data/

In [ ]:
import csv

df = pd.read_csv(
    "./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz",
    delimiter="\t",
    quoting=csv.QUOTE_NONE,
    compression="gzip",
)
df.shape

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

df[["star_rating", "review_id"]].groupby("star_rating").count().plot(kind="bar", title="Breakdown by Star Rating")
plt.xlabel("Star Rating")
plt.ylabel("Review Count")

## Balance the Dataset

In [ ]:
print("Shape of dataframe before splitting {}".format(df.shape))

In [ ]:
# Balance the dataset down to the minority class
df_grouped_by = df.groupby(["star_rating"]) 
df_balanced = df_grouped_by.apply(lambda x: x.sample(df_grouped_by.size().min()).reset_index(drop=True))

df_balanced = df_balanced.reset_index(drop=True)
print("Shape of balanced dataframe {}".format(df_balanced.shape))

In [ ]:
df_balanced[["star_rating", "review_id"]].groupby("star_rating").count().plot(
    kind="bar", title="Breakdown by Star Rating"
)
plt.xlabel("Star Rating")
plt.ylabel("Review Count")

## Split the Data into Train, Validation, and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

# Split all data into 90% train and 10% holdout
df_train, df_holdout = train_test_split(df_balanced, test_size=0.10, stratify=df_balanced["star_rating"])

# Split holdout data into 50% validation and 50% test
df_validation, df_test = train_test_split(df_holdout, test_size=0.50, stratify=df_holdout["star_rating"])

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = ["Train", "Validation", "Test"]
sizes = [len(df_train.index), len(df_validation.index), len(df_test.index)]
explode = (0.1, 0, 0)

fig1, ax1 = plt.subplots()

ax1.pie(sizes, explode=explode, labels=labels, autopct="%1.1f%%", startangle=90)

# Equal aspect ratio ensures that pie is drawn as a circle.
ax1.axis("equal")

plt.show()

## Write a Train CSV with Header for Autopilot

In [ ]:
autopilot_train_path = "./amazon_reviews_us_Digital_Software_v1_00_autopilot.csv"
df_train.to_csv(autopilot_train_path, index=False, header=True)

## Upload Train Data to S3 for Autopilot

In [ ]:
train_s3_prefix = "data"
autopilot_train_s3_uri = sess.upload_data(path=autopilot_train_path, key_prefix=train_s3_prefix)
autopilot_train_s3_uri

In [ ]:
!aws s3 ls $autopilot_train_s3_uri

## Write a CSV with no header for Amazon Comprehend

In [ ]:
noheader_train_path = "./amazon_reviews_us_Digital_Software_v1_00_noheader.csv"
df_train.to_csv(noheader_train_path, index=False, header=False)

## Upload Train Data to S3 for Comprehend

In [ ]:
train_s3_prefix = "data"
noheader_train_path = sess.upload_data(path=noheader_train_path, key_prefix=train_s3_prefix)

## Store Variables for Next Notebook(s)

In [ ]:
%store autopilot_train_s3_uri

In [ ]:
%store noheader_train_s3_uri

# Train a Model with SageMaker Autopilot

I used Autopilot to predict the star rating of customer reviews. Autopilot implements a transparent approach to AutoML.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

## See prepared training data which I used as input for Autopilot

In [ ]:
import csv

df = pd.read_csv("./tmp/amazon_reviews_us_Digital_Software_v1_00_autopilot.csv")
df.head()

## Setup the S3 Location for the Autopilot-Generated Assets

This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Training Models.

In [ ]:
prefix_model_output = "models/autopilot"

model_output_s3_uri = "s3://{}/{}".format(bucket, prefix_model_output)

print(model_output_s3_uri)

### Configure Autopilot

In [ ]:
max_candidates = 3

job_config = {
    "CompletionCriteria": {
        "MaxRuntimePerTrainingJobInSeconds": 900,
        "MaxCandidates": max_candidates,
        "MaxAutoMLJobRuntimeInSeconds": 5400,
    },
}

input_data_config = [
    {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": "{}".format(autopilot_train_s3_uri)}},
        "TargetAttributeName": "star_rating",
    }
]

output_data_config = {"S3OutputPath": "{}".format(model_output_s3_uri)}


## Launch the SageMaker Autopilot Job

In [ ]:
from time import gmtime, strftime, sleep

In [ ]:
%store -r auto_ml_job_name

try:
    auto_ml_job_name
except NameError:
    timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
    auto_ml_job_name = "automl-dm-" + timestamp_suffix
    print("Created AutoMLJobName: " + auto_ml_job_name)

In [ ]:
%store auto_ml_job_name

In [ ]:
max_running_jobs = 1

if running_jobs < max_running_jobs:  # Limiting to max. 1 Jobs
    try:
        sm.create_auto_ml_job(
            AutoMLJobName=auto_ml_job_name,
            InputDataConfig=input_data_config,
            OutputDataConfig=output_data_config,
            AutoMLJobConfig=job_config,
            RoleArn=role,
        )
        print("[OK] Autopilot Job {} created.".format(auto_ml_job_name))
        running_jobs = running_jobs + 1
    except:
        print(
            "[INFO] You have already launched an Autopilot job. Please continue see the output of this job.".format(
                running_jobs
            )
        )
else:
    print(
        "[INFO] You have already launched {} Autopilot running job(s). Please continue see the output of the running job.".format(
            running_jobs
        )
    )

## Track the Progress of the Autopilot Job

SageMaker Autopilot job consists of the following high-level steps:

* Data Analysis where the data is summarized and analyzed to determine which feature engineering techniques, hyper-parameters, and models to explore.
* Feature Engineering where the data is scrubbed, balanced, combined, and split into train and validation.
* Model Training and Tuning where the top performing features, hyper-parameters, and models are selected and trained.

## Analyzing Data and Generate Notebooks

In [ ]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while (
    "AutoMLJobStatus" not in job_description_response.keys()
    and "AutoMLJobSecondaryStatus" not in job_description_response.keys()
):
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job has not yet started. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for Autopilot Job to start...")
    sleep(15)

print("[OK] AutoMLJob started.")

## Review the SageMaker Processing Jobs

* First Processing Job (Data Splitter) checks the data sanity, performs stratified shuffling and splits the data into training and validation.
* Second Processing Job (Candidate Generator) first streams through the data to compute statistics for the dataset. Then, uses these statistics to identify the problem type, and possible types of every column-predictor: numeric, categorical, natural language, etc.

Once data analysis is complete, SageMaker AutoPilot generates two notebooks:

* Data Exploration

* Candidate Definition


In [ ]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while "AutoMLJobArtifacts" not in job_description_response.keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job has not yet generated the artifacts. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for AutoMLJobArtifacts...")
    sleep(15)

print("[OK] AutoMLJobArtifacts generated.")

In [ ]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while "DataExplorationNotebookLocation" not in job_description_response["AutoMLJobArtifacts"].keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job has not yet generated the notebooks. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for DataExplorationNotebookLocation...")
    sleep(15)

print("[OK] DataExplorationNotebookLocation found.")

In [ ]:
generated_resources = job_description_response["AutoMLJobArtifacts"]["DataExplorationNotebookLocation"]
download_path = generated_resources.rsplit("/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb")[0]
job_id = download_path.rsplit("/", 1)[-1]

## Feature Engineering

Watch out for SageMaker Training Jobs and Batch Transform Jobs to start.

* This is the candidate exploration phase.
* Each python script code for data-processing is executed inside a SageMaker framework container as a training job, followed by transform job.

Feature preprocessing part of each pipeline has all hyper parameters fixed, i.e. does not require tuning, thus feature preprocessing step can be done prior runing the hyper parameter optimization job.

It outputs up to 10 variants of transformed data, therefore algorithms for each pipeline are set to use the respective transformed data.

In [ ]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
print(job_status)
print(job_sec_status)
if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("FeatureEngineering"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Feature engineering phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

## Model Training and Tuning

Watch out for a SageMakerHyperparameter Tuning Job and various Training Jobs to start.

* All algorithms are optimized using a SageMaker Hyperparameter Tuning job.
* Up to 250 training jobs (based on number of candidates specified) are selectively executed to find the best candidate model.

In [ ]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
print(job_status)
print(job_sec_status)
if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("ModelTuning"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Model tuning phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

### Viewing All Candidates
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [ ]:
candidates_response = sm.list_candidates_for_auto_ml_job(
    AutoMLJobName=auto_ml_job_name, SortBy="FinalObjectiveMetricValue"
)

### Inspect Trials using Experiments API
SageMaker Autopilot automatically creates a new experiment, and pushes information for each trial.


In [ ]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=auto_ml_job_name + "-aws-auto-ml-job",
)

df = exp.dataframe()
print(df)

### Explore the Best Candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using Inference Pipelines. For this notebook, I deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [ ]:
best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

In [ ]:
while "BestCandidate" not in best_candidate_response:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job is generating BestCandidate. Please wait. ")
    print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate generated.")

In [ ]:
print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))

In [ ]:
while "CandidateName" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateName. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate CandidateName generated.")

In [ ]:
while "FinalAutoMLJobObjectiveMetric" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate FinalAutoMLJobObjectiveMetric. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.")

In [ ]:
best_candidate_identifier = best_candidate["CandidateName"]
print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"])
print("Metric value: " + str(best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"]))

In [ ]:
print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))

### Autopilot Chooses XGBoost as Best Candidate!
Note that Autopilot chose different hyper-parameters and feature transformations than we used in our own XGBoost model.

### Deploy the Model as a REST Endpoint

Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [ ]:
%store -r autopilot_model_name

In [ ]:
try:
    autopilot_model_name
except NameError:
    timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
    autopilot_model_name = "automl-dm-model-" + timestamp_suffix
    print("[OK] Created Autopilot Model Name: " + autopilot_model_name)

In [ ]:
%store autopilot_model_name

In [ ]:
%store -r autopilot_model_arn

In [ ]:
try:
    autopilot_model_arn
except NameError:
    create_model_response = sm.create_model(
        Containers=best_candidate["InferenceContainers"], ModelName=autopilot_model_name, ExecutionRoleArn=role
    )
    autopilot_model_arn = create_model_response["ModelArn"]
    print("[OK] Created Autopilot Model: {}".format(autopilot_model_arn))

In [ ]:
%store autopilot_model_arn

### Define EndpointConfig Name

In [ ]:
timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
epc_name = "automl-dm-epc-" + timestamp_suffix

print(epc_name)

In [ ]:
variant_name = "automl-dm-variant-" + timestamp_suffix
print("[OK] Created Endpoint Variant Name {}: ".format(variant_name))

In [ ]:
%store autopilot_endpoint_name

In [ ]:
ep_config = sm.create_endpoint_config(
    EndpointConfigName=epc_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
            "ModelName": autopilot_model_name,
            "VariantName": variant_name,
        }
    ],
)

In [ ]:
%store -r autopilot_endpoint_arn

In [ ]:
try:
    autopilot_endpoint_arn
except NameError:
    create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_name, EndpointConfigName=epc_name)
    autopilot_endpoint_arn = create_endpoint_response["EndpointArn"]
    print(autopilot_endpoint_arn)

In [ ]:
%store autopilot_endpoint_arn

### Wait for the Model to Deploy

In [ ]:
sm.get_waiter("endpoint_in_service").wait(EndpointName=autopilot_endpoint_name)

In [ ]:
resp = sm.describe_endpoint(EndpointName=autopilot_endpoint_name)
status = resp["EndpointStatus"]

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [ ]:
sm_runtime = boto3.client("sagemaker-runtime")

In [ ]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(
    EndpointName=autopilot_endpoint_name, ContentType="text/csv", Accept="text/csv", Body=csv_line_predict_positive
)

response_body = response["Body"].read().decode("utf-8").strip()

r = response_body.split(",")
print("Predicated Star Rating Class: {} \nProbability: {} ".format(r[0], r[1]))

In [ ]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(
    EndpointName=autopilot_endpoint_name, ContentType="text/csv", Accept="text/csv", Body=csv_line_predict_meh
)

response_body = response["Body"].read().decode("utf-8").strip()

r = response_body.split(",")
print("Predicated Star Rating Class: {} \nProbability: {} ".format(r[0], r[1]))

In [ ]:
csv_line_predict_negative = """It's pretty good."""

response = sm_runtime.invoke_endpoint(
    EndpointName=autopilot_endpoint_name, ContentType="text/csv", Accept="text/csv", Body=csv_line_predict_negative
)

response_body = response["Body"].read().decode("utf-8").strip()

r = response_body.split(",")
print("Predicated Star Rating Class: {} \nProbability: {} ".format(r[0], r[1]))
